In [ ]:
from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
import time, math
import tensorflow_probability as tfp
tfd = tfp.distributions

tf.reset_default_graph()
tf.set_random_seed(777)

xy = np.loadtxt("X8 data input path", delimiter=',', dtype=np.float32)

xy_data = xy.reshape(-1,11)
np.random.shuffle(xy_data)

data_size = len(xy_data)

input_dim = 8
z_dim = 5
output_dim = 16

x_data_training = xy_data[:, 0:-3]
y_data_training = xy_data[:, -3:-1]
y_1 = xy_data[:, -1]
y_1=tf.one_hot(y_1, 3)

batch_size = tf.placeholder(tf.int32, name = 'batch_size')
X = tf.placeholder(tf.float32, shape=[None, input_dim])
learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')


In [ ]:
#Encoder

W1 = tf.get_variable(name='weight1', shape=[input_dim, 64], initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable(name='weight2', shape=[64, 32], initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable(name='weight3', shape=[32, 16], initializer=tf.contrib.layers.xavier_initializer())
W4_u = tf.get_variable(name='weight4_u', shape=[16, z_dim], initializer=tf.contrib.layers.xavier_initializer())

b1 = tf.Variable(tf.random_normal([64]), name='bias1')
b2 = tf.Variable(tf.random_normal([32]), name='bias2')
b3 = tf.Variable(tf.random_normal([16]), name='bias3')
b4_u = tf.Variable(tf.random_normal([z_dim]), name='bias4_u')

L2 = tf.matmul(X, W1) + b1
L3 = tf.matmul(L2, W2) + b2
L4 = tf.matmul(L3, W3) + b3
L5_u = tf.matmul(L4, W4_u) + b4_u

one = tf.ones([batch_size,1])
one = tf.reshape(one, [batch_size])

Sigma_square_mat = tf.concat([tf.transpose([tf.exp(L5_u[:,2])]),tf.transpose([tf.exp(L5_u[:,3])]), 
                           tf.transpose([tf.exp(L5_u[:,3])]),tf.transpose([tf.exp(L5_u[:,4])])], 1)

Sigma_square_mat = tf.reshape(Sigma_square_mat, shape=[batch_size, 2, 2])
Sigma_mat = tf.sqrt(Sigma_square_mat)

eps = tf.random_normal(([batch_size, 2, 1]), 0, 1, dtype=tf.float32)

eps = tf.reshape(tf.matmul(Sigma_mat, eps),[batch_size,2])

Z = tf.add(L5_u[:,:2], eps) 
Z_ = tf.concat([Z, y_1],1)


In [ ]:
# Decoder

W1_d = tf.get_variable(name='weight1_d', shape=[2, 16], initializer=tf.contrib.layers.xavier_initializer())
W2_d = tf.get_variable(name='weight2_d', shape=[16, 32], initializer=tf.contrib.layers.xavier_initializer())
W3_d = tf.get_variable(name='weight3_d', shape=[32, 64], initializer=tf.contrib.layers.xavier_initializer())
W4_d_u = tf.get_variable(name='weight4_d_u', shape=[64, output_dim], initializer=tf.contrib.layers.xavier_initializer())

b1_d = tf.Variable(tf.random_normal([16]), name='bias1_d')
b2_d = tf.Variable(tf.random_normal([32]), name='bias2_d')
b3_d = tf.Variable(tf.random_normal([64]), name='bias3_d')
b4_d_u = tf.Variable(tf.random_normal([output_dim]), name='bias4_d_u')


L2_d = tf.matmul(Z, W1_d) + b1_d
L3_d = tf.matmul(L2_d, W2_d) + b2_d
L4_d = tf.matmul(L3_d, W3_d) + b3_d
L5_d_u = tf.matmul(L4_d, W4_d_u) + b4_d_u

one_ = tf.ones([batch_size,8])

eps_d = tf.random_normal(([batch_size, 8]), 0, 1, dtype=tf.float32)
X_ = tf.add(L5_d_u[:,:8], tf.multiply(tf.sqrt(tf.exp(L5_d_u[:,8:])), eps_d))


In [ ]:
Cov_i_1 = tf.fill([batch_size, 1],3.0432)
Cov_i_2 = tf.fill([batch_size, 1],-2.49357)
Cov_i_3 = tf.fill([batch_size, 1],-2.49357)
Cov_i_4 = tf.fill([batch_size, 1],3.0432)
Cov_inv_b =  tf.concat([Cov_i_1, Cov_i_2, Cov_i_3, Cov_i_4], 1)
Cov_inv_b = tf.reshape(Cov_inv_b, shape=[batch_size, 2, 2])
trr = tf.matmul(Cov_inv_b,tf.matmul(Sigma_mat,Sigma_mat))

KL_loss_1 = tf.trace(trr)
condition = tf.less(KL_loss_1, 0.)
KL_loss_1  = tf.where(condition, tf.math.negative(KL_loss_1), KL_loss_1)

Cov_inv = tf.constant([[3.0432, -2.49357], 
                       [-2.49357, 3.0432]])
L5_U1 = tf.fill([batch_size, 1],0.)
L5_U2 = tf.fill([batch_size, 1],0.)
L5_Uo = tf.concat([L5_U1, L5_U2], 1)

KL_loss_2 = tf.reduce_sum(tf.multiply(tf.matmul((L5_Uo - L5_u[:,0:2]),Cov_inv),(L5_Uo - L5_u[:,0:2])),1)

Cov = tf.constant([[1., 0.819389255], 
                   [0.819389255, 1.]])
det = tf.linalg.det(Cov)
det_cov_up = tf.fill([batch_size],det)

det_cov_down = tf.linalg.det(tf.matmul(Sigma_mat,Sigma_mat))

condition = tf.less(det_cov_down, 0.)
dum = tf.ones([batch_size])
det_cov_down_  = tf.where(condition, dum*0.0000000001, det_cov_down)
KL_loss_3 = tf.math.log(tf.div_no_nan(det_cov_up, det_cov_down_))
        
KL_loss = KL_loss_1 + KL_loss_2 + KL_loss_3

Repro_loss = tf.reduce_sum(0.5 * tf.math.log(tf.exp(L5_d_u[:,8:]))
                   + tf.div_no_nan(tf.square(X-L5_d_u[:,:8]), 2.0 * tf.exp(L5_d_u[:,8:])), 1)

cost =  tf.reduce_mean(Repro_loss) + tf.reduce_mean(KL_loss - 2)*0.5
optimizer =  tf.train.AdamOptimizer(learning_rate_).minimize(cost)

In [ ]:

init = tf.global_variables_initializer()
saver = tf.train.Saver()


with tf.Session() as sess:

    print("\nStart training")
    sess.run(init)
    batch_size_ = 500
    for epoch in range(1000):
        total_batch = int(data_size / batch_size_)
        if epoch < 500:
            LR = 0.01
        if epoch > 500:
            LR = 0.01

        for i in range(total_batch):
            batch_x = x_data_training[i*batch_size_:(i+1)*batch_size_]
            _, d, Z_val = sess.run((optimizer, cost, L5_u), feed_dict={X: batch_x, batch_size: batch_size_, learning_rate_: LR})
    
            if i % 50 == 0:
                print("Epoch #%d step=%d cost=%f" % (epoch, i, d))
   
    save_path = saver.save(sess, "checkpoint path") 
    Z_val, Z_, X_mean_val, X_d, eps_ = sess.run((L5_u, Z, L5_d_u, X_, eps_d), feed_dict={X: x_data_training[:data_size,:], batch_size: data_size, learning_rate_: LR})

    plt.plot( Z_[:,0], Z_[:,1], '+')
    plt.plot(y_data_training[:data_size,0], y_data_training[:data_size, 1], '.')
    plt.show()

for i in range(7):
    plt.plot(X_d[:,0], X_d[:,i+1], '+')
    plt.plot(x_data_training[:data_size,0], x_data_training[:data_size,i+1], '*')
    plt.show()
for i in range(6):
    plt.plot(X_d[:,1], X_d[:,i+2], '+')
    plt.plot(x_data_training[:data_size,1], x_data_training[:data_size,i+2], '*')
    plt.show()
for i in range(5):
    plt.plot(X_d[:,2], X_d[:,i+3], '+')
    plt.plot(x_data_training[:data_size,2], x_data_training[:data_size,i+3], '*')
    plt.show()
for i in range(4):
    plt.plot(X_d[:,3], X_d[:,i+4], '+')
    plt.plot(x_data_training[:data_size,3], x_data_training[:data_size,i+4], '*')
    plt.show()
for i in range(3):
    plt.plot(X_d[:,4], X_d[:,i+5], '+')
    plt.plot(x_data_training[:data_size,4], x_data_training[:data_size,i+5], '*')
    plt.show()
for i in range(2):
    plt.plot(X_d[:,5], X_d[:,i+6], '+')
    plt.plot(x_data_training[:data_size,5], x_data_training[:data_size,i+6], '*')
    plt.show()
for i in range(1):
    plt.plot(X_d[:,6], X_d[:,i+7], '+')
    plt.plot(x_data_training[:data_size,6], x_data_training[:data_size,i+7], '*')
    plt.show()
for i in range(0):
    plt.plot(X_d[:,7], X_d[:,i+8], '+')
    plt.plot(x_data_training[:data_size,7], x_data_training[:data_size,i+8], '*')
    plt.show()